In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
sys.path.insert(0, "../../scripts")
from utils import load_data


from pycytominer.cyto_utils import infer_cp_features


import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from sklearn.decomposition import PCA
from tensorflow import keras

from vae import VAE

from tensorflow.keras.models import Model, Sequential
import seaborn
import random as python_random
import tensorflow as tf
# import umap

In [3]:
data_splits = ["train", "test", "valid", "complete"]
data_dict = load_data(data_splits)

/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [4]:
# Prepare data for training
meta_features = infer_cp_features(data_dict["train"], metadata=True)
cp_features = infer_cp_features(data_dict["train"])

train_features_df = data_dict["train"].reindex(cp_features, axis="columns")
train_meta_df = data_dict["train"].reindex(meta_features, axis="columns")

test_features_df = data_dict["test"].reindex(cp_features, axis="columns")
test_meta_df = data_dict["test"].reindex(meta_features, axis="columns")

valid_features_df = data_dict["valid"].reindex(cp_features, axis="columns")
valid_meta_df = data_dict["valid"].reindex(meta_features, axis="columns")

complete_features_df = data_dict["complete"].reindex(cp_features, axis="columns")
complete_meta_df = data_dict["complete"].reindex(meta_features, axis="columns")


In [5]:
print(train_features_df.shape)
train_features_df.head(3)

(40242, 685)


,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_MedianRadius,Cells_AreaShape_Orientation,Cells_AreaShape_Solidity,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,...,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_Mito_5_0,Nuclei_Texture_SumEntropy_RNA_20_0,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_SumVariance_DNA_20_0,Nuclei_Texture_SumVariance_RNA_20_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0
0,0.19790,0.69605,0.40400,0.30834,0.19317,0.71895,0.38837,0.38668,0.62494,0.87362,...,0.61045,0.41907,0.64945,0.65199,0.22660,0.32108,0.29776,0.26173,0.34524,0.32151
1,0.18605,0.68873,0.41051,0.31982,0.19885,0.72510,0.39552,0.38362,0.65094,0.89679,...,0.61393,0.38269,0.63814,0.68204,0.23700,0.32808,0.27981,0.26893,0.34938,0.33179
2,0.19083,0.66276,0.38585,0.31437,0.20415,0.71380,0.38701,0.36699,0.60932,0.88492,...,0.60850,0.41850,0.63529,0.65208,0.22852,0.32710,0.29371,0.27159,0.33871,0.31965


In [6]:
def shuffle_each_column(df):
    columns = df.columns
    df_copy = df.copy()
    for column in columns:
        df_copy[column] = df_copy[column].sample(frac=1).reset_index(drop=True)
    return (df_copy)

In [7]:
train_features_df = shuffle_each_column(train_features_df)
valid_features_df = shuffle_each_column(valid_features_df)

In [8]:
encoder_architecture=[250]
decoder_architecture=[250]

In [9]:
cp_vae = VAE(
    input_dim=train_features_df.shape[1],
    latent_dim=90,
    batch_size=32,
    encoder_batch_norm=True,
    epochs=50,
    learning_rate=0.0001,
    encoder_architecture=encoder_architecture,
    decoder_architecture=decoder_architecture,
    beta=0.06,
    verbose=True,
)
cp_vae.compile_vae()

2022-01-14 02:10:07.379031: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-14 02:10:07.379082: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-14 02:10:07.379127: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-3-81): /proc/driver/nvidia/version does not exist
2022-01-14 02:10:07.379474: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/keras/optimizer_v

In [10]:
cp_vae.train(x_train=train_features_df, x_test=valid_features_df)

2022-01-14 02:10:09.096960: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
1258/1258 [==============================] - 11s 7ms/step - loss: 54.8923 - recon: 53.5778 - kl: 1.3144 - mmd: 0.0000e+00 - val_loss: 12.1777 - val_recon: 10.9528 - val_kl: 1.2248 - val_mmd: 0.0000e+00
Epoch 2/50
1258/1258 [==============================] - 8s 6ms/step - loss: 8.3507 - recon: 7.3164 - kl: 1.0343 - mmd: 0.0000e+00 - val_loss: 6.6617 - val_recon: 5.6243 - val_kl: 1.0374 - val_mmd: 0.0000e+00
Epoch 3/50
1258/1258 [==============================] - 8s 6ms/step - loss: 5.1262 - recon: 4.3243 - kl: 0.8020 - mmd: 0.0000e+00 - val_loss: 4.5070 - val_recon: 3.7012 - val_kl: 0.8058 - val_mmd: 0.0000e+00
Epoch 4/50
1258/1258 [==============================] - 8s 6ms/step - loss: 3.4647 - recon: 2.8889 - kl: 0.5757 - mmd: 0.0000e+00 - val_loss: 2.8894 - val_recon: 2.3861 - val_kl: 0.5032 - val_mmd: 0.0000e+00
Epoch 5/50
1258/1258 [==============================] - 8s 6ms/step - loss: 2.4404 - recon: 2.0658 - kl: 0.3747 - mmd: 0.0000e+00 - val_loss: 2.1286 - val_recon: 1

Epoch 40/50
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0393 - recon: 1.0392 - kl: 1.7287e-05 - mmd: 0.0000e+00 - val_loss: 0.9917 - val_recon: 0.9914 - val_kl: 2.8662e-04 - val_mmd: 0.0000e+00
Epoch 41/50
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0392 - recon: 1.0391 - kl: 1.6137e-05 - mmd: 0.0000e+00 - val_loss: 0.9902 - val_recon: 0.9900 - val_kl: 2.1700e-04 - val_mmd: 0.0000e+00
Epoch 42/50
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0391 - recon: 1.0390 - kl: 1.5350e-05 - mmd: 0.0000e+00 - val_loss: 0.9904 - val_recon: 0.9902 - val_kl: 1.5502e-04 - val_mmd: 0.0000e+00
Epoch 43/50
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0390 - recon: 1.0390 - kl: 1.4347e-05 - mmd: 0.0000e+00 - val_loss: 0.9917 - val_recon: 0.9916 - val_kl: 1.2268e-04 - val_mmd: 0.0000e+00
Epoch 44/50
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0389 - recon: 1.0389 - kl: 1.3935e-05 - mmd: 0.0000e+

In [11]:
cp_vae.vae

In [12]:
# Save training performance
history_df = pd.DataFrame(cp_vae.vae.history.history)
history_df

,loss,recon,kl,mmd,val_loss,val_recon,val_kl,val_mmd
0,54.892296,53.577824,1.314364,0.0,12.177666,10.952815,1.224849,0.0
1,8.350660,7.316357,1.034306,0.0,6.661676,5.624257,1.037420,0.0
2,5.126224,4.324258,0.801971,0.0,4.507024,3.701198,0.805828,0.0
3,3.464654,2.888922,0.575735,0.0,2.889389,2.386144,0.503246,0.0
4,2.440433,2.065762,0.374671,0.0,2.128606,1.792644,0.335962,0.0
5,1.823447,1.609350,0.214099,0.0,1.583832,1.406919,0.176913,0.0
6,1.460065,1.350541,0.109524,0.0,1.317115,1.222072,0.095043,0.0
7,1.269654,1.213995,0.055659,0.0,1.171761,1.124651,0.047110,0.0
8,1.172160,1.139324,0.032835,0.0,1.103393,1.067432,0.035960,0.0
9,1.120640,1.098759,0.021882,0.0,1.061594,1.038763,0.022832,0.0


In [13]:
encoder = cp_vae.encoder_block["encoder"]
decoder = cp_vae.decoder_block["decoder"]
encoder.save("models/level4EncoderShuffled_beta")
decoder.save("models/level4DecoderShuffled_beta")

2022-01-14 02:16:45.787950: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/level4EncoderShuffled_beta/assets
INFO:tensorflow:Assets written to: models/level4DecoderShuffled_beta/assets


In [14]:
# history_df.to_csv('level4_training_random.csv')

In [15]:
# history_df = pd.read_csv('level4_training_random.csv')

In [16]:
# original_training_data  = pd.read_csv('level4_training.csv')

In [17]:
# plt.figure(figsize=(7, 5), dpi = 400)
# plt.plot(original_training_data["loss"], label="Training data")
# plt.plot(original_training_data["val_loss"], label="Validation data")
# plt.plot(history_df["loss"], label="Shuffled training data")
# plt.plot(history_df["val_loss"], label="Shuffled validation data")
# # plt.title("Loss for VAE training on Cell Painting Level 5 data")
# plt.ylabel("MSE + KL Divergence")
# plt.xlabel("No. Epoch")
# plt.ylim(0,5)
# plt.legend()
# plt.show()

In [18]:
cp_vae = VAE(
    input_dim=train_features_df.shape[1],
    latent_dim=90,
    batch_size=32,
    encoder_batch_norm=True,
    epochs=58,
    learning_rate=0.0001,
    encoder_architecture=encoder_architecture,
    decoder_architecture=decoder_architecture,
    beta=1,
    verbose=True,
)
cp_vae.compile_vae()

/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
cp_vae.train(x_train=train_features_df, x_test=valid_features_df)

Epoch 1/58
1258/1258 [==============================] - 11s 6ms/step - loss: 70.3704 - recon: 52.7581 - kl: 17.6124 - mmd: 0.0000e+00 - val_loss: 26.5960 - val_recon: 10.9781 - val_kl: 15.6180 - val_mmd: 0.0000e+00
Epoch 2/58
1258/1258 [==============================] - 8s 6ms/step - loss: 12.5164 - recon: 6.5975 - kl: 5.9189 - mmd: 0.0000e+00 - val_loss: 8.1625 - val_recon: 4.6823 - val_kl: 3.4802 - val_mmd: 0.0000e+00
Epoch 3/58
1258/1258 [==============================] - 8s 6ms/step - loss: 5.2896 - recon: 3.8543 - kl: 1.4354 - mmd: 0.0000e+00 - val_loss: 4.0096 - val_recon: 3.0823 - val_kl: 0.9273 - val_mmd: 0.0000e+00
Epoch 4/58
1258/1258 [==============================] - 8s 6ms/step - loss: 3.1639 - recon: 2.7058 - kl: 0.4581 - mmd: 0.0000e+00 - val_loss: 2.7240 - val_recon: 2.2602 - val_kl: 0.4638 - val_mmd: 0.0000e+00
Epoch 5/58
1258/1258 [==============================] - 8s 6ms/step - loss: 2.2694 - recon: 2.0354 - kl: 0.2340 - mmd: 0.0000e+00 - val_loss: 2.0295 - val_recon

1258/1258 [==============================] - 8s 6ms/step - loss: 1.0401 - recon: 1.0401 - kl: 1.0280e-06 - mmd: 0.0000e+00 - val_loss: 0.9921 - val_recon: 0.9920 - val_kl: 6.3616e-05 - val_mmd: 0.0000e+00
Epoch 40/58
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0400 - recon: 1.0400 - kl: 1.0544e-06 - mmd: 0.0000e+00 - val_loss: 0.9919 - val_recon: 0.9919 - val_kl: 3.2176e-05 - val_mmd: 0.0000e+00
Epoch 41/58
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0399 - recon: 1.0399 - kl: 9.3989e-07 - mmd: 0.0000e+00 - val_loss: 0.9917 - val_recon: 0.9917 - val_kl: 4.9938e-05 - val_mmd: 0.0000e+00
Epoch 42/58
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0399 - recon: 1.0399 - kl: 8.8792e-07 - mmd: 0.0000e+00 - val_loss: 0.9906 - val_recon: 0.9906 - val_kl: 1.2076e-05 - val_mmd: 0.0000e+00
Epoch 43/58
1258/1258 [==============================] - 8s 6ms/step - loss: 1.0396 - recon: 1.0396 - kl: 8.4685e-07 - mmd: 0.0000e+00 - val_los

In [20]:
cp_vae.vae

In [21]:
# Save training performance
history_df = pd.DataFrame(cp_vae.vae.history.history)
history_df

,loss,recon,kl,mmd,val_loss,val_recon,val_kl,val_mmd
0,70.370407,52.758076,1.761240e+01,0.0,26.596024,10.978057,15.617961,0.0
1,12.516389,6.597512,5.918881e+00,0.0,8.162502,4.682303,3.480198,0.0
2,5.289619,3.854258,1.435357e+00,0.0,4.009625,3.082335,0.927292,0.0
3,3.163886,2.705787,4.581015e-01,0.0,2.723989,2.260216,0.463773,0.0
4,2.269424,2.035388,2.340383e-01,0.0,2.029506,1.751306,0.278200,0.0
5,1.760998,1.626251,1.347477e-01,0.0,1.617525,1.445691,0.171833,0.0
6,1.453811,1.382861,7.095001e-02,0.0,1.391315,1.251676,0.139638,0.0
7,1.272382,1.240887,3.149613e-02,0.0,1.555143,1.150310,0.404834,0.0
8,1.169523,1.158463,1.106157e-02,0.0,3.104126,1.110373,1.993753,0.0
9,1.113103,1.109824,3.279684e-03,0.0,1.231197,1.045007,0.186190,0.0


In [22]:
encoder = cp_vae.encoder_block["encoder"]
decoder = cp_vae.decoder_block["decoder"]
encoder.save("models/level4EncoderShuffled_vanilla")
decoder.save("models/level4DecoderShuffled_vanilla")

INFO:tensorflow:Assets written to: models/level4EncoderShuffled_vanilla/assets
INFO:tensorflow:Assets written to: models/level4DecoderShuffled_vanilla/assets


In [23]:
# history_df.to_csv('level4_training_vanilla_random.csv')

In [24]:
# history_df = pd.read_csv('level4_training_vanilla_random.csv')

In [25]:
# original_training_data  = pd.read_csv('level4_training_vanilla.csv')

In [26]:
# plt.figure(figsize=(7, 5), dpi = 400)
# plt.plot(original_training_data["loss"], label="Training data")
# plt.plot(original_training_data["val_loss"], label="Validation data")
# plt.plot(history_df["loss"], label="Shuffled training data")
# plt.plot(history_df["val_loss"], label="Shuffled validation data")
# # plt.title("Loss for VAE training on Cell Painting Level 5 data")
# plt.ylabel("MSE + KL Divergence")
# plt.xlabel("No. Epoch")
# plt.ylim(0,5)
# plt.legend()
# plt.show()

In [27]:
cp_vae = VAE(
    input_dim=train_features_df.shape[1],
    latent_dim=90,
    batch_size=32,
    encoder_batch_norm=True,
    epochs=50,
    learning_rate=0.0001,
    encoder_architecture=encoder_architecture,
    decoder_architecture=decoder_architecture,
    beta=0,
    lam = 10000,
    verbose=True,
)
cp_vae.compile_vae()

/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [28]:
cp_vae.train(x_train=train_features_df, x_test=valid_features_df)

Epoch 1/50
1258/1258 [==============================] - 10s 6ms/step - loss: 78.0448 - recon: 54.4351 - kl: 0.0000e+00 - mmd: 23.5357 - val_loss: 35.0460 - val_recon: 11.1319 - val_kl: 0.0000e+00 - val_mmd: 24.5580
Epoch 2/50
1258/1258 [==============================] - 8s 6ms/step - loss: 25.3812 - recon: 7.2241 - kl: 0.0000e+00 - mmd: 18.1660 - val_loss: 24.9239 - val_recon: 5.2630 - val_kl: 0.0000e+00 - val_mmd: 20.2246
Epoch 3/50
1258/1258 [==============================] - 8s 6ms/step - loss: 19.8233 - recon: 4.1960 - kl: 0.0000e+00 - mmd: 15.5104 - val_loss: 22.2547 - val_recon: 3.4452 - val_kl: 0.0000e+00 - val_mmd: 19.2425
Epoch 4/50
1258/1258 [==============================] - 8s 6ms/step - loss: 17.1954 - recon: 2.8252 - kl: 0.0000e+00 - mmd: 14.3655 - val_loss: 18.1752 - val_recon: 2.3540 - val_kl: 0.0000e+00 - val_mmd: 15.9632
Epoch 5/50
1258/1258 [==============================] - 8s 6ms/step - loss: 15.8905 - recon: 2.0554 - kl: 0.0000e+00 - mmd: 13.8377 - val_loss: 18.57

1258/1258 [==============================] - 8s 6ms/step - loss: 11.8520 - recon: 1.0242 - kl: 0.0000e+00 - mmd: 10.9194 - val_loss: 14.8790 - val_recon: 0.9754 - val_kl: 0.0000e+00 - val_mmd: 13.9525
Epoch 40/50
1258/1258 [==============================] - 8s 6ms/step - loss: 11.8183 - recon: 1.0236 - kl: 0.0000e+00 - mmd: 10.8040 - val_loss: 14.3923 - val_recon: 0.9757 - val_kl: 0.0000e+00 - val_mmd: 13.6228
Epoch 41/50
1258/1258 [==============================] - 8s 6ms/step - loss: 11.7689 - recon: 1.0227 - kl: 0.0000e+00 - mmd: 10.7524 - val_loss: 13.8829 - val_recon: 0.9760 - val_kl: 0.0000e+00 - val_mmd: 13.1357
Epoch 42/50
1258/1258 [==============================] - 8s 6ms/step - loss: 11.6395 - recon: 1.0218 - kl: 0.0000e+00 - mmd: 10.6704 - val_loss: 14.2639 - val_recon: 0.9738 - val_kl: 0.0000e+00 - val_mmd: 13.3096
Epoch 43/50
1258/1258 [==============================] - 8s 6ms/step - loss: 11.6389 - recon: 1.0211 - kl: 0.0000e+00 - mmd: 10.6509 - val_loss: 14.6947 - val_r

In [29]:
cp_vae.vae

In [30]:
# Save training performance
history_df = pd.DataFrame(cp_vae.vae.history.history)
history_df

,loss,recon,kl,mmd,val_loss,val_recon,val_kl,val_mmd
0,78.044777,54.435055,0.0,23.535709,35.046013,11.131903,0.0,24.557991
1,25.381241,7.224068,0.0,18.166002,24.923874,5.262980,0.0,20.224573
2,19.823282,4.196005,0.0,15.510356,22.254669,3.445248,0.0,19.242504
3,17.195419,2.825225,0.0,14.365484,18.175192,2.354046,0.0,15.963177
4,15.890542,2.055385,0.0,13.837684,18.572666,1.778565,0.0,16.531254
5,15.104353,1.613759,0.0,13.533289,17.325323,1.431351,0.0,16.166536
6,14.632801,1.362445,0.0,13.330157,17.436863,1.240074,0.0,16.453779
7,14.437928,1.221852,0.0,13.212083,16.539963,1.127990,0.0,16.311323
8,14.133565,1.140476,0.0,13.019581,19.453928,1.068129,0.0,18.436792
9,14.070037,1.094904,0.0,12.894222,16.728477,1.034456,0.0,16.167183


In [31]:
encoder = cp_vae.encoder_block["encoder"]
decoder = cp_vae.decoder_block["decoder"]
encoder.save("models/level4EncoderShuffled_mmd")
decoder.save("models/level4DecoderShuffled_mmd")

INFO:tensorflow:Assets written to: models/level4EncoderShuffled_mmd/assets
INFO:tensorflow:Assets written to: models/level4DecoderShuffled_mmd/assets


In [32]:
# history_df.to_csv('level4_training_mmd_random.csv')

In [33]:
# original_training_data  = pd.read_csv('level4_training_mmd.csv')

In [34]:
# plt.figure(figsize=(7, 5), dpi = 400)
# plt.plot(original_training_data["loss"], label="Training data")
# plt.plot(original_training_data["val_loss"], label="Validation data")
# plt.plot(history_df["loss"], label="Shuffled training data")
# plt.plot(history_df["val_loss"], label="Shuffled validation data")
# # plt.title("Loss for VAE training on Cell Painting Level 5 data")
# plt.ylabel("MSE + MMD")
# plt.xlabel("No. Epoch")
# plt.legend()
# plt.show()